In [93]:
import sys

sys.path.append(r'C:\users\17049\appdata\local\programs\python\python311\lib\site-packages')

import pandas as pd
import numpy as np
import fredapi as fa
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Get debt data from https://fred.stlouisfed.org/
fred = fa.Fred(api_key =os.environ.get('FRED_API_KEY'))

#Format data
d = fred.get_series('GFDEBTN')
d.name = 'Total US Public Debt'
dfd = pd.DataFrame(d).reset_index()
dfd.rename(columns={'index':'Date'}, inplace=True)
dfd['Date'] = pd.to_datetime(dfd['Date'], format='%Y-%m-%d')
dfd['Total US Public Debt'] = dfd['Total US Public Debt'].astype(float)/1000000
dfd['Last Name'] = np.NaN

#Import president data
pres = pd.read_excel(r'C:\Users\17049\Documents\Data\Presidents.xlsx', header=1)

#Filter and format data
pres = pres[pres['Number'] > 35]
pres['Time in Office (ms)'] = (pres['End'] - pres['Start'])/np.timedelta64(1,'ms')
pres.rename(columns={'Start':'Date'}, inplace=True)
pres['Date'] = pd.to_datetime(pres['Date'], format='%Y-%m-%d')
pres['Total US Public Debt'] = np.NaN
pres['Marker'] = 1

#Merge and fill 
m = pd.concat([dfd, pres], axis=0).sort_values('Date')
m['Last Name'].fillna(method='ffill', inplace=True)
m['Total US Public Debt'].fillna(method='ffill', inplace=True)
m['Number'].fillna(method='ffill', inplace=True)
m['Marker'].fillna(method='ffill', inplace=True)
m['Bar Color'] = np.where(m['Number'] % 2 == 0, 'white', 'grey')
m['Time in Office (ms)'].fillna(method='ffill', inplace=True)

m.tail()

,Date,Total US Public Debt,Last Name,Number,End,Time in Office (ms),Marker,Bar Color
225,2022-04-01,30.568582,Biden,46.0,NaN,1.262304e+11,1.0,white
226,2022-07-01,30.928912,Biden,46.0,NaN,1.262304e+11,1.0,white
227,2022-10-01,31.419689,Biden,46.0,NaN,1.262304e+11,1.0,white
228,2023-01-01,31.458438,Biden,46.0,NaN,1.262304e+11,1.0,white
229,2023-04-01,32.332274,Biden,46.0,NaN,1.262304e+11,1.0,white


In [92]:
#Construct chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=m['Date'], y=m['Total US Public Debt'], mode='lines', name='Total US Public Debt ($ T)',
                         marker_color='red', yaxis='y'))

fig.add_trace(go.Bar(name='President in Office', x=m['Date'], y=m['Marker'], marker_color=m['Bar Color'],offset=0,
                     width=m['Time in Office (ms)'], opacity=0.6, yaxis='y2'))

fig.update_layout(barmode='group', title_text='Total US Public Debt', showlegend=False,
                  hoverlabel=dict(bgcolor='white'), template='plotly_white',  
                  xaxis=dict(showgrid=False, ticklabelmode='period'),
                  yaxis1=dict(title='Total Public Debt ($ T)',tickformat=',.1f',range=[0,35], zeroline=False,showgrid=True),
                  yaxis2=dict(showticklabels=False, zeroline=False, showgrid=False,range=[0,1]))

#Annotations
fig.add_annotation(text='Source: https://fred.stlouisfed.org/',
                  xref='paper', yref='paper',
                  x=0.95, y=-0.15, showarrow=False)

fig.add_annotation(x='2025-01-01', y=30, text=f'Aug 2023<br>$32.8 T', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='grey'), align='center')

fig.add_annotation(x='1966-01-20', y=33, text=f'Johnson', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='1971-06-1', y=33, text=f'Nixon', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='1975-04-20', y=33, text=f'Ford', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='1979-01-20', y=33, text=f'Carter', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='1985-01-20', y=33, text=f'Reagan', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='1991-01-20', y=33, text=f'Bush', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='1997-01-20', y=33, text=f'Clinton', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='2005-01-20', y=33, text=f'Bush (W)', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='2013-01-20', y=33, text=f'Obama', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='2019-01-20', y=33, text=f'Trump', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')
fig.add_annotation(x='2023-01-20', y=33, text=f'Biden', yanchor='bottom', showarrow=False,
                   font=dict(size=10, color='black'), align='center')

fig.show()

In [94]:
fig.write_html(r'C:\Users\17049\Documents\Charts\USPublicDebt.html', full_html=False, include_plotlyjs='cdn')